# 2. Similitud entre productos

- Objetivo: El objetivo de este desafío es desarrollar un análisis que permita medir la similitud entre títulos de productos en Mercado Libre Brasil, utilizando el dataset “item_titles_test.csv”. A través de técnicas de procesamiento de lenguaje natural, se busca identificar y listar los pares de títulos más similares, ordenados por un score de similitud, facilitando así la agrupación automática de productos similares.


este problema se puede resolver de varias formas. Opté por usar embeddings y ChromaBD por su facilidad de implementación y efectividad al atacar este tipo de problemas, donde a asociación está atada a qué tan parecido un producto puede ser dado su titulación dentro de Mercado Libre.

El enfoque por el que de decidí optar es realizar una comparación de similitud semántica del título de los productos permitiendo fácil su agrupación según un criterio de similitud.

## Implementación con Embeddings usando Sentence-Transformers.

In [1]:
# Importación de Librerías
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions

In [ ]:
# --- Carga de datos ---
file_path = r'..\data\items_titles.csv'
column_name = 'ITE_ITEM_TITLE'

try:
    df_items = pd.read_csv(file_path, header=0)
    if column_name in df_items.columns:
        documents = df_items[column_name].dropna().astype(str).tolist()
        # Crear IDs únicos para cada documento
        ids = [str(i) for i in range(len(documents))]
        print(f"Se cargaron {len(documents)} títulos del archivo '{file_path}'.")
    else:
        print(f"Error: La columna '{column_name}' no se encontró.")
        documents, ids = [], []
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta '{file_path}'.")
    documents, ids = [], []


# --- Inicialización de ChromaDB ---

client = chromadb.PersistentClient(path="./chroma_db_meli")

# Modelo multilingüe, adecuado para portugués y español.
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="paraphrase-multilingual-MiniLM-L12-v2"
)

collection_name = "meli_products"
print(f"\nCargando o creando la colección de ChromaDB: '{collection_name}'")
collection = client.get_or_create_collection(
    name=collection_name,
    metadata={"embedding_function": "paraphrase-multilingual-MiniLM-L12-v2"}
)

print("Cliente y colección de ChromaDB listos.")

Se cargaron 30000 títulos del archivo '..\data\items_titles.csv'.

Cargando o creando la colección de ChromaDB: 'meli_products'
Cliente y colección de ChromaDB listos.


In [3]:
from tqdm.notebook import tqdm
import time


# 1. Obtener los IDs de TODOS los documentos que ya existen en la colección.
try:
    existing_ids = set(collection.get(include=[])['ids'])
    print(f"Se encontraron {len(existing_ids)} items existentes en la colección.")
except Exception as e:
    print(f"No se pudo obtener la lista de IDs (la colección podría ser nueva). Error: {e}")
    existing_ids = set()

# 2. Determinar qué documentos faltan por añadir.
docs_to_add = [doc for doc, doc_id in zip(documents, ids) if doc_id not in existing_ids]
ids_to_add = [doc_id for doc_id in ids if doc_id not in existing_ids]

# 3. Proceder con la carga solo si hay documentos pendientes.
if not docs_to_add:
    print("\n¡La colección ya está completa y actualizada! No hay nada que añadir.")
else:
    print(f"\nFaltan por añadir {len(docs_to_add)} documentos. Iniciando proceso de carga.")
    
    batch_size = 256
    
    for i in tqdm(range(0, len(docs_to_add), batch_size), desc="Reanudando carga en ChromaDB"):
        batch_docs = docs_to_add[i:i+batch_size]
        batch_ids = ids_to_add[i:i+batch_size]
        
        # Añadir el lote a la colección.
        collection.add(
            documents=batch_docs,
            ids=batch_ids
        )

print(f"\nProceso finalizado. Total de items en la colección: {collection.count()}")

Se encontraron 30000 items existentes en la colección.

¡La colección ya está completa y actualizada! No hay nada que añadir.

Proceso finalizado. Total de items en la colección: 30000


In [28]:
def find_most_similar_chroma(query_title: str, top_k: int, collection: chromadb.Collection):
    """
    Encuentra los 'top_k' títulos más similares a un 'query_title' usando ChromaDB.

    Args:
        query_title (str): El título para el cual buscar similitudes.
        top_k (int): El número de resultados a devolver.
        collection (chromadb.Collection): La colección de ChromaDB a consultar.

    Returns:
        list: Una lista de tuplas (título_similar, score_de_similitud).
    """
    # La colección se encarga de generar el embedding, ChromaDB lo hace internamente.
    results = collection.query(
        query_texts=[query_title],
        n_results=top_k + 1 # Uno más para descartar el mismo título, donde la similitud es 1.0
    )
    
    similar_items = []

    result_docs = results['documents'][0]
    result_dists = results['distances'][0]
    
    for doc, dist in zip(result_docs, result_dists):
        # Descartar el mismo título (distancia 0)
        if doc != query_title:
            # El 'distance' de Chroma es una medida de distancia (0 = idéntico).
            # Para tener un 'score de similitud' (1 = idéntico)
            similarity_score = 1 - dist
            similar_items.append((doc, similarity_score))
            
    return similar_items[:top_k]

def comparar_productos(product_to_check, top_n, collection):
    """
    Busca los productos más similares a uno dado y retorna un DataFrame ordenado con el producto consultado,
    el producto similar y el score de similitud.

    Args:
        product_to_check (str): Producto a consultar.
        top_n (int): Número de similares a mostrar.
        collection (chromadb.Collection): Colección de ChromaDB.

    Returns:
        pd.DataFrame: DataFrame con columnas ['Producto Consultado', 'Título Similar', 'Score de Similitud']
    """
    df_similares = find_most_similar_chroma(product_to_check, top_n, collection)
    df = pd.DataFrame(df_similares, columns=['Título Similar', 'Score de Similitud'])
    df.insert(0, 'Producto Consultado', product_to_check)
    return df

In [29]:
if documents:
    product_to_check = 'Tênis Casual Feminino Moleca Tecido Tie Dye'
    top_n = 15

    print(f"\n--- Buscando los {top_n} productos más similares a: '{product_to_check}' ---")
    df = comparar_productos(product_to_check, top_n, collection)

df


--- Buscando los 15 productos más similares a: 'Tênis Casual Feminino Moleca Tecido Tie Dye' ---


,Producto Consultado,Título Similar,Score de Similitud
0,Tênis Casual Feminino Moleca Tecido Tie Dye,Tênis Feminino Tie Dye Casual Colorido Calce F...,0.842622
1,Tênis Casual Feminino Moleca Tecido Tie Dye,Tênis Feminino Colorido Tecido Tie Dye,0.839842
2,Tênis Casual Feminino Moleca Tecido Tie Dye,Tênis Feminino Moleca Multicolor Tie Dye,0.838187
3,Tênis Casual Feminino Moleca Tecido Tie Dye,Tenis Feminino Casual Plataforma Tie Dye,0.825386
4,Tênis Casual Feminino Moleca Tecido Tie Dye,Tênis Casual Feminino Tie Dye Com Cadarço,0.820465
5,Tênis Casual Feminino Moleca Tecido Tie Dye,Tênis Feminino Casual Tie Dye Acompanha Brind...,0.758021
6,Tênis Casual Feminino Moleca Tecido Tie Dye,Tênis Tie Dye Feminino Casual,0.741461
7,Tênis Casual Feminino Moleca Tecido Tie Dye,Tênis Feminino Flatform Casual Tanara Tie Dye,0.737218
8,Tênis Casual Feminino Moleca Tecido Tie Dye,Tênis Feminino Casual Branco Sapatenis Tie Dy...,0.728560
9,Tênis Casual Feminino Moleca Tecido Tie Dye,Tênis Feminino Mulheres Crianças All Star Tie ...,0.726922


In [30]:
if documents:
    product_to_check = 'Bicicleta Tsw Evo Quest Gx 12v 29x19 Az Metalico'
    top_n = 15

    print(f"\n--- Buscando los {top_n} productos más similares a: '{product_to_check}' ---")
    df = comparar_productos(product_to_check, top_n, collection)

df


--- Buscando los 15 productos más similares a: 'Bicicleta Tsw Evo Quest Gx 12v 29x19 Az Metalico' ---


,Producto Consultado,Título Similar,Score de Similitud
0,Bicicleta Tsw Evo Quest Gx 12v 29x19 Az Metalico,Bicicleta Tsw Evo Quest Gx 12v 29x19 Az Metalico,1.000000
1,Bicicleta Tsw Evo Quest Gx 12v 29x19 Az Metalico,Bicicleta Aro 29 Tsw Evo Quest Gx 12v,0.748341
2,Bicicleta Tsw Evo Quest Gx 12v 29x19 Az Metalico,Bicicleta 29 Evo-quest Advanced-gx - Tsw,0.532010
3,Bicicleta Tsw Evo Quest Gx 12v 29x19 Az Metalico,Bicicleta Tsw Evo Quest Advanced Gx Carbono 29...,0.506272
4,Bicicleta Tsw Evo Quest Gx 12v 29x19 Az Metalico,Bicicleta Ciclismo Mtb Tsw Evo Quest Carbon 29...,0.456008
5,Bicicleta Tsw Evo Quest Gx 12v 29x19 Az Metalico,Bicicleta Mtb Aro 29 Quadro 19 21v Z7x Amarelo...,0.416877
6,Bicicleta Tsw Evo Quest Gx 12v 29x19 Az Metalico,Bicicleta Mtb Aro 29 Soul Sl329 Eagle Sx 12v A...,0.407771
7,Bicicleta Tsw Evo Quest Gx 12v 29x19 Az Metalico,Bicicleta Mtb Aro 29 Soul Sl329 Eagle Sx 12v B...,0.395297
8,Bicicleta Tsw Evo Quest Gx 12v 29x19 Az Metalico,Bicicleta Tsw Ride 21v 29x17 Cz/lj Nac.,0.377563
9,Bicicleta Tsw Evo Quest Gx 12v 29x19 Az Metalico,Bicicleta Aro 29 Z7-x 27v 19,0.352576


# Conclusiones y Hallazgos

## Justificación del Modelo y Tecnologías Utilizadas

### Modelo Sentence-Transformer Multilingüe
La elección del modelo `paraphrase-multilingual-MiniLM-L12-v2` se fundamenta en varios aspectos clave:

1. **Capacidad Multilingüe**: Dado que Mercado Libre opera en varios países con lenguas diferentes, pero puede contener productos con nombres en español o incluso inglés, un modelo multilingüe garantiza una comprensión semántica robusta independientemente del idioma del título.

2. **Optimización para Paráfrasis**: Este modelo está específicamente entrenado para identificar similitudes semánticas entre textos que pueden expresar la misma idea con diferentes palabras, ideal para títulos de productos que pueden describir el mismo artículo de maneras distintas.

3. **Eficiencia Computacional**: La variante MiniLM ofrece un buen balance entre precisión y velocidad, permitiendo procesar grandes volúmenes de datos sin comprometer significativamente la calidad de los embeddings. Sin embargo, para texto con mayor longitud o complejidad, como las propias descripciones de los productos, se sugiere usar un embedding mas robusto que permita una cadena de entrada mayor.

### ChromaDB como Base de Datos Vectorial
La implementación con ChromaDB proporciona ventajas significativas:

1. **Búsqueda Vectorial Eficiente**: Permite realizar consultas de similitud rápidas sobre grandes conjuntos de datos.

2. **Persistencia**: Los embeddings se almacenan persistentemente, evitando recálculos innecesarios en ejecuciones posteriores.

3. **Escalabilidad**: Facilita la adición incremental de nuevos productos sin necesidad de reconstruir toda la base de datos.

4. **Simplicidad de Implementación**: Integración directa con modelos de Sentence-Transformers, reduciendo la complejidad del código.

## Enfoque Semántico vs. Métodos Alternativos

### Ventajas del Enfoque Semántico Elegido

La comparación semántica mediante embeddings ofrece robustez:

1. **Resilencia a Errores Ortográficos**: Los embeddings capturan el significado subyacente, por lo que pequeñas variaciones ortográficas no afectan significativamente la similitud calculada.

2. **Manejo de Sinónimos**: Identifica productos similares aunque usen vocabulario diferente (ej: "tênis" vs "sapato esportivo").

3. **Comprensión Contextual**: Entiende el contexto completo del título.

4. **Invarianza a Orden de Palabras**: La similitud se mantiene independientemente del orden en que aparezcan las características del producto.

### Métodos Alternativos Considerados

Existen otros enfoques para abordar este problema, cada uno con sus limitaciones:

1. **Similitud de Cadenas (String Similarity)**:
   - Métodos: Levenshtein, Jaccard, Cosine similarity con TF-IDF
   - Limitaciones: Sensibles a errores ortográficos y variaciones en la estructura del título

2. **Expresiones Regulares y Patrones**:
   - Enfoque: Extracción de características específicas (marca, modelo, color)
   - Limitaciones: Requiere reglas manuales extensas y es poco esacalable. 


## Resultados Obtenidos

Los experimentos realizados demuestran la efectividad del enfoque:

1. **Alta Precisión Semántica**: Los productos más similares mostrados son conceptualmente relacionados, indicando que el modelo captura correctamente las relaciones semánticas.

2. **Diversidad en los Resultados**: El sistema identifica similitudes explícitas (misma marca/modelo) como sutiles (productos de categorías relacionadas).

## Recomendaciones para Implementación en Producción

1. **Umbral de Similitud**: Establecer un umbral mínimo de similitud (ej: 0.6) para filtrar asociaciones débiles y no solo traer una cantidad concreta sino todo el grupo que cumpla el criterio.

2. **Actualización Incremental**: Implementar un pipeline para añadir nuevos productos sin recalcular embeddings existentes.

3. **Monitoreo de Calidad**: Establecer métricas de evaluación para desvirtuar o fortalecer las agrupaciones sugeridads.

4. **Optimización de Hiperparámetros**: Se podría llegar a considerar fine-tuning del modelo con datos específicos de Mercado Libre para mejorar la alienación con negocio.